# Importing modules

In [4]:
import numpy as np
import time
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from sklearn.preprocessing import normalize
import properscoring as ps

# Constructing Sigma Matrix

In [5]:
# defining parameter to be considered
param = 'Wy'

In [6]:
#load or save the file with the forecast (+6 hours)
file_save = ('Wind_UVT-level_20000-NForc_006-May')
directory = 'data/'
filename = directory + file_save + '.npz'
print(filename)
data = np.load(filename)
Wx_2 = data[param]
time_2 = data['vec_time']
hour_2 = data['delta_hours']
data.close()

data/Wind_UVT-level_20000-NForc_006-May.npz


In [7]:
#because the downloaded data had a problem when being parsed
time_2 = time_2[:-1]

In [8]:
#defining the mean of all the sigmas 
sigma_mean = np.mean(np.std(Wx_2,3),2)

In [10]:
#load or save the file with the values of sigma_mean for each latitude and longitude
file_save = 'sigma_mean_' + param
directory = 'data/'
filename = directory + file_save + '.npz'
np.savez(filename, sigma_mean = sigma_mean)
#data = np.load(filename)
#sigma_mean = data['sigma_mean']
print(filename)
data.close()

data/sigma_mean_Wy.npz


# Sigma validation

## {Wx, data points: 12510720, 192 data steps, $\sigma$ estimated in 124 time steps.}

In [12]:
#load or save the file with the observed data
file_save = '2018_0456-Wind_UVT-level_20000-1Forc_000'
directory = 'data/'
filename = directory + file_save + '.npz'
#np.savez(filename, Wx_alpha = Wx_15min)
#print(filename)
data = np.load(filename)
Wx_0 = data[param]
time_0 = data['vec_time']
hour_0 = data['delta_hours']
data.close()

In [13]:
#load or save the file with the forecasted data
file_save = '2018_0456-Wind_UVT-level_20000-1Forc_006'
directory = 'data/'
filename = directory + file_save + '.npz'
#np.savez(filename, Wx_alpha = Wx_15min)
#print(filename)
data = np.load(filename)
Wx_1 = data[param]
time_1 = data['vec_time']
hour_1 = data['delta_hours']
data.close()

In [ ]:
#Chooses the data so they will be in the same time_step
time0 = []
ii = []
jj = []
for i in range(len(time_0)):
    for j in range(len(time_1)):
        if time_0[i] == time_1[j]:
            time0 += [time_0[i]]
            ii += [i]
            jj += [j]

tim_0 = time_0[ii]
tim_1 = time_1[jj]
Wx_0 = Wx_0[:,:,ii]
Wx_1 = Wx_1[:,:,jj]
delta_hours = (tim_0[1:] - tim_1[:-1])
n = len(delta_hours)
dhour = np.zeros([n,1])
for i in range(0,n):
    dhour[i] = delta_hours[i].total_seconds()/3600

In [ ]:
#73 - 197 covers the same dates as the ones used to calculate sigma

Result0 = []
for i in range(73):
    Result0 += [ps.crps_gaussian(Wx_0[:,:,i], mu=Wx_1[:,:,i], sig=sigma_mean)]
for i in range(197,len(time0)):
    Result0 += [ps.crps_gaussian(Wx_0[:,:,i], mu=Wx_1[:,:,i], sig=sigma_mean)]
    
print(np.sum(Result0))
print(np.mean(Result0))

## {Wx, data points: 19 222 200, 295 data steps, $\sigma$ estimated in 20 time steps. }

In [ ]:
sigma_mean1 = np.mean(np.std(Wx_2[:,:,:20],3),2)
sigma_mean1.shape

In [ ]:
Result0 = []
for i in range(73):
    Result0 += [ps.crps_gaussian(Wx_0[:,:,i], mu=Wx_1[:,:,i], sig=sigma_mean1)]
for i in range(73+20,len(time0)):
    Result0 += [ps.crps_gaussian(Wx_0[:,:,i], mu=Wx_1[:,:,i], sig=sigma_mean1)]
    
print(np.sum(Result0))
print(np.mean(Result0))

## {Wx, data points: 7 102 440, 99 data steps, $\sigma$ estimated in 20 time steps. }

In [ ]:
#load or save the file with the observed data
file_save = '2018_0456-Wind_UVT-level_20000-1Forc_000'
directory = 'data_matrix/'
filename = directory + file_save + '.npz'
#np.savez(filename, Wx_alpha = Wx_15min)
#print(filename)
data = np.load(filename)
Wx_0 = data[param]
time_0 = data['vec_time']
hour_0 = data['delta_hours']
data.close()

In [ ]:
#load or save the file with the forecasted data
file_save = '2018_0456-Wind_UVT-level_20000-1Forc_006'
directory = 'data_matrix/'
filename = directory + file_save + '.npz'
#np.savez(filename, Wx_alpha = Wx_15min)
#print(filename)
data = np.load(filename)
Wx_1 = data[param]
time_1 = data['vec_time']
hour_1 = data['delta_hours']
data.close()

In [ ]:
#Chooses the data so they will be in the same time_step
time0 = []
ii = []
jj = []
kk = []
for i in range(len(time_0)):
    for j in range(len(time_1)):
        for k in range(len(time_2)):
            if time_0[i] == time_1[j]:
                if time_0[i] == time_2[k]:
                    time0 += [time_0[i]]
                    ii += [i]
                    jj += [j]
                    kk += [k]

tim_0 = time_0[ii]
tim_1 = time_1[jj]
Wx_0 = Wx_0[:,:,ii]
Wx_1 = Wx_1[:,:,jj]
Wx_2 = Wx_2[:,:,kk,:]
delta_hours = (tim_0[1:] - tim_1[:-1])
n = len(delta_hours)
dhour = np.zeros([n,1])
for i in range(0,n):
    dhour[i] = delta_hours[i].total_seconds()/3600

In [ ]:
Result0 = []
Result1 = []
for i in range(20,len(time0)):
#for i in range(1):
    Result0 += [ps.crps_gaussian(Wx_0[:,:,i], mu=Wx_1[:,:,i], sig=sigma_mean1)]
    Result1 += [ps.crps_ensemble(Wx_0[:,:,i], Wx_2[:,:,i,:])]
    

print('Gaussian!')
print(np.sum(Result0))
print(np.mean(Result0))
print('Ensemble!')
print(np.sum(Result1))
print(np.mean(Result1))